Notebook author: Makar Shevchenko B20-DS-01

# Model Training and Visualization. Movie Recommender System. Assignment 2. Practical Machine Learning and Deep Learning

## Solution

### Imports

### Config

### Modelling

#### Model Selection

#### Test Design

#### Model Building

### Evaluation